In [ ]:
# Library ----------------------------------------------------------------
library(dplyr)
library(tidyr)
library(lubridate)

# 1. Load Data, Clustering Results and Scale Vectors ---------------------------

kmeans_result <- readRDS("../../../data/qc2022/preparation_donnees/03_pilot1_kmeans.rds")

appData <- readRDS("data/qc2022/preparation_donnees/01_app_2022.rds")

# pour process_app_data
orig_center <- readRDS("kmeans_orig_center.rds") 
orig_scale  <- readRDS("kmeans_orig_scale.rds")

Est-ce que les données de l'app ont été scale?
Si non, fonction pour process_app_data

In [ ]:
# Fonction pour assigner cluster
assign_clusters <- function(data, centroids) {
  # Convertir en matrices
  data <- as.matrix(data)
  centroids <- as.matrix(centroids)
  
  # Initialiser la matrice des distances
  distances <- matrix(NA, nrow = nrow(data), ncol = nrow(centroids))
  
  # Calculer les distances Euclidiennes
  for (k in seq_len(nrow(centroids))) {
    diff <- sweep(data, 2, centroids[k, ], FUN = "-")
    distances[, k] <- rowSums(diff^2)  # somme des différences au carré
  }
  
  # Assigner le cluster le plus proche
  cluster_assignment <- apply(distances, 1, which.min)
  return(cluster_assignment)
}

In [ ]:
write_rds(cluster_assignment, file = "data/qc2022/preparation_donnees/05_app_2022_clustered.rds")